# Numerical studies with the TIEGCM to characterize tidal effects on the longitudinal patterns of PRE


- This code generates a .nc file that is used in the PRE_2024_Paper_Figures.ipynb notebook.
- Note: this took several tens of GB of RAM on my machine.


In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import apexpy
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
from scipy import interpolate

import gc

import sys
import time as time_module
from IPython.display import display, clear_output
def printerase(s):
    clear_output(wait=True)
    time_module.sleep(0.01)
    print(s)
    sys.stdout.flush()

import dynamo


#### Various helper functions

In [ ]:
def get_solar_zenith_angle(t, lat, lon, alt):
    '''
    Calculate the angle from zenith to the sun at the time and
    location specified.
    INPUT:
        t - python datetime (assumed to be UT)
        lat - Latitude, deg
        lon - Longitude, deg
        alt - Altitude, km
    OUTPUT:
        sza - Solar zenith angle (deg)
    '''
    import ephem
    import numpy as np
    sun = ephem.Sun()
    obs = ephem.Observer()
    obs.lon = '%f' % (lon)
    obs.lat = '%f' % (lat)
    obs.date = t.strftime('%Y/%m/%d %H:%M:%S')
    obs.pressure = 0. # ignore refraction. This makes a negligible difference.
    obs.elevation = 1000*alt # This makes a negligible difference.
    sun.compute(obs)
    sza = np.pi/2 - float(sun.alt) # radians
    return 180/np.pi * sza # degrees

In [ ]:
def view_TIEGCM_by_slt(dsday):
    '''
    For a standard TIEGCM file, return a new Dataset, organized by LT instead of geog longitude.
    Assumes variable 'lon' and 'time'.
    
    Use primitive definition of LT for convenience.
    
    Note this doesn't affect high-latitude potential, since that is in geomag coordinates. So to be
    safe it sets it to NaN
    '''
    sltg = np.arange(0,24, 24/len(dsday.lon)) # Grid of LTs
    # For each timestamp, shift data in lon
    dss = []
    for n in range(len(dsday.time)):
        ds = dsday.isel(time=n)
        ds['slt'] = np.mod(ds.time.dt.hour + ds.lon/15., 24)
        ds = ds.swap_dims({'lon':'slt'}).sortby('slt')
        assert (ds['slt']-sltg).std() < 1e-2
        ds['slt'] = sltg # Avoid rounding errors by imposing grid 
        dss.append(ds)
    dsday2 = xr.concat(dss, dim='time') # lon reindexed by LT
    
    
    return dsday2

In [ ]:
def view_TIEGCM_by_lon(dsday2):
    '''
    For a TIEGCM file organized by LT, return a new Dataset, organized by geog lon instead of LT.
    Assumes variable 'slt' and 'time'.
    
    Use primitive definition of lon-LT conversion, for convenience
    '''
    long = np.arange(-180,180, 360/len(dsday2.slt)) # Grid of LTs
    # For each timestamp, shift data in LT
    dss = []
    for n in range(len(dsday2.time)):
        ds = dsday2.isel(time=n)
        ds['lon'] = np.mod(180 + 15*(ds.slt - ds.time.dt.hour), 360) - 180.
        ds = ds.swap_dims({'slt':'lon'}).sortby('lon')
        assert (ds['lon']-long).std() < 1e-2
        ds['lon'] = long # Avoid rounding errors by imposing grid 
        dss.append(ds)
    dsday3 = xr.concat(dss, dim='time') # LT reindexed by lon
    dsday3 = dsday3.drop('slt') # Make sure this vestige doesn't stick around as a coordinate
    return dsday3


In [ ]:
def setup_using_TIEGCM_vars(d, ds, hilat='tiegcm', extrap_tiegcm_down=False, ingest_wind=True, ingest_cond=True):
    '''
    This is like "solve_using_TIEGCM_vars" (which I'm omitting from the v2 nb to reduce confusion) 
    except it doesn't do the compute_FLI & solve steps.
    It also makes sure to save the hlb and llb so they can be used in later steps.
    
    Take an intialized dynamo object "d", fill in winds and conductivities using a TIEGCM dataset "ds"
    (organized the standard way by lon, not by LT). "ds" should be a single timestamp.
    
    Note this modifies "d" in place.
    
    hilat = 'zero': Use zero potential at upper and lower boundary
          = 'tiegcm': Use potential from TIEGCM. (Will be taken from N hemis, which could be different
                      than S if mlatde_max is too large)
                      
    extrap_tiegcm_down = True (default): Extrap the grid down to ~82 km (exponentially for conductivity and linearly for winds)
                         False: Keep the bottom at ~98 km. Any grid points below this will be zero-order extrapolated. (Not
                                recommended, but slightly faster)
                   
    ingest_wind = whether to use TIEGCM wind. If not, the variable isn't even initialized
    ingest_cond = whether to use TIEGCM cond. If not, the variable isn't even initialized
                   
    Note: I tried to generalize this to allow multiple times at once, but ran into interpolation trouble.
          Just stick with looping over single timestamps for now.
          
    After doing this, you should be able to call compute_FLI and solve.
    '''
    
    # Warn if altitude extrapolation is too much
    if d.alt[0] < 95e3 and not extrap_tiegcm_down:
        print('WARNING: Minimum alt gridpoint is %.1f km. Are you sure you want extrap_tiegcm_down=False?' % (d.alt[0]/1e3))
    

    ds['ZGlev'] = ds.ZG.interp(ilev=ds.lev).assign_coords(lev=ds.lev)

    ds2 = ds[['SIGMA_HAL','SIGMA_PED','UN','VN','ZGlev']] # Only save certain variables that need to be interpolated
    ds2 = ds2.isel(lev=slice(None,-1)) # Top alt is nan so just ignore it.

    # Protect for -180/180 crossover in the interpolation. Copy the last lon to the end of the file
    ds2x = ds2.isel(lon=0).assign_coords(lon=180.) # masquerade lon = -180 as +180
    ds3 = xr.concat((ds2, ds2x), dim='lon')
    
    # Optional extrapolation down to ~82 km.
    if extrap_tiegcm_down:
        lev = ds3.lev.values
        dlev = np.diff(lev)[0] # 0.25
        N = 13 # How many points to extrapolate. Using 13 gets down to ~82 km.
        n = np.arange(N,0,-1)
        lev2 = np.concatenate((lev[0]-dlev*n, lev))
        
        # Replace conductivities with their log, so that interpolation is exponential
        for v in ['SIGMA_HAL','SIGMA_PED']:
            ds3[v] = np.log(ds3[v])

        ds3 = ds3.interp(lev=lev2).interpolate_na(dim='lev', fill_value='extrapolate', method='linear')
        # Change conductivity back to real value
        for v in ['SIGMA_HAL','SIGMA_PED']:
            ds3[v] = np.exp(ds3[v])

    # Find the "lev" coordinates for each dynamo grid
    altN = ds3.ZGlev.interp(lat=d.GLATDN, lon=d.GLONDN)/1e2 # m
    altS = ds3.ZGlev.interp(lat=d.GLATDS, lon=d.GLONDS)/1e2 # m

    diffS = altS - d.alt
    diffN = altN - d.alt
    topS = (diffS<=0).all(dim='lev') # Alt is above the TIEGCM grid
    topN = (diffN<=0).all(dim='lev') # Alt is above the TIEGCM grid
    botS = (diffS>=0).all(dim='lev') # Alt is below
    botN = (diffN>=0).all(dim='lev') # Alt is below
    midS = ~topS & ~botS & ~d.GLATDN.isnull() # Alt is in the middle
    midN = ~topN & ~botN & ~d.GLATDN.isnull() # Alt is in the middle
    iS0 = diffS.where(diffS<0 & midS).fillna(value=-np.inf).argmax(dim='lev')
    iN0 = diffN.where(diffN<0 & midN).fillna(value=-np.inf).argmax(dim='lev')
    iS0 = iS0.where(midS, other=0) # Only fill in the "mid" points
    iN0 = iN0.where(midN, other=0)

    levS0 = ds3.lev.isel(lev=iS0)
    levN0 = ds3.lev.isel(lev=iN0)
    levS1 = ds3.lev.isel(lev=iS0+1)
    levN1 = ds3.lev.isel(lev=iN0+1)
    altS0 = altS.isel(lev=iS0)
    altN0 = altN.isel(lev=iN0)
    altS1 = altS.isel(lev=iS0+1)
    altN1 = altN.isel(lev=iN0+1)
    levS = levS0 + (levS1 - levS0)/(altS1 - altS0)*(d.alt - altS0)
    levN = levN0 + (levN1 - levN0)/(altN1 - altN0)*(d.alt - altN0)
    # Handle top and bottom cases
    levS.values[topS.values] = ds3.lev[-1]
    levN.values[topN.values] = ds3.lev[-1]
    levS.values[botS.values] = ds3.lev[0] # TODO: Think about extrapolating lower if you want to test electrojet stuff
    levN.values[botN.values] = ds3.lev[0]

    # Interpolate the TIEGCM variables onto the dynamo grid
    dsiN = ds3.interp(lev=levN, lat=d.GLATDN, lon=d.GLONDN)
    dsiS = ds3.interp(lev=levS, lat=d.GLATDS, lon=d.GLONDS)

    # Ingest wind and conductivity
    if ingest_cond:
        d['sigPN'] = (['mlat','mlon','alt'], dsiN.SIGMA_PED.values)
        d['sigPS'] = (['mlat','mlon','alt'], dsiS.SIGMA_PED.values)
        d['sigHN'] = (['mlat','mlon','alt'], dsiN.SIGMA_HAL.values)
        d['sigHS'] = (['mlat','mlon','alt'], dsiS.SIGMA_HAL.values)
    if ingest_wind:
        d['ugN'] = (['mlat','mlon','alt','vec2'], np.stack((dsiN.UN.values, dsiN.VN.values), axis=3)/1e2)
        d['ugS'] = (['mlat','mlon','alt','vec2'], np.stack((dsiS.UN.values, dsiS.VN.values), axis=3)/1e2)

    # OPTIONS for high-latitude boundary values
    if hilat == 'zero':
        hlb = 0. # Zero
    elif hilat == 'tiegcm':
        phi_gcm_N = ds.PHIM2D.sel(mlat= d.mlatde[-1], method='nearest') # Taking closest match. Avoid tolerance, ignore N/S match.
        # phi_gcm_S = ds.PHIM2D.sel(mlat=-d.mlatde[-1], method='nearest', tolerance=0.01)
        # assert((phi_gcm_N - phi_gcm_S).std() < 1e-2)
        hlb = phi_gcm_N.interp(mlon = d.mlonde).values # Interpolate to dynamo grid. Note TIEGCM mlon grid already has ghost cell

    if isinstance(hlb,str) and np.array_equal(hlb, 'Kl=0'):
        d['hlb']  = (['mlon'], np.nan*np.zeros(d.N))
    else:
        d['hlb']  = (['mlon'], hlb)
    # We are always using the "no-current" lower boundary condition
    d['llb']  = (['mlon'], np.nan*np.zeros(d.N))


    # Slight hack: Re-initalize time variables to be consistent with TIEGCM file
    d.attrs['tref'] = pd.to_datetime(ds.time.item())
    d['mlt'] = (['mlon'], d.apex_obj.mlon2mlt(d.mlond, d.tref))
    _, lon_at_mageq_at_hR = d.apex_obj.convert(0, d.mlond, 'apex', 'geo', height=d.hR/1e3, )
    d['slteq'] = (['mlon'], np.array([dynamo.compute_slt(d.tref, lon) for lon in lon_at_mageq_at_hR]))

## Initialize and fill in Dynamo objects 

In [ ]:
hR = 90e3 # reference height for apex coordinates
mlatde_max = 55. # Just low/mid-latitude

# OPTION 1: Good resolution
mlonde_res = 7.5 
mlatde_res = 3.5
zskip = 1
skip = 2 # for TIEGCM ingestion, choose 1 to average all days (watch memory!) Otherwise downsample for speed/testing. skip=2 is recommended (32 GB on my machine)


# OPTION 2: Low resolution, just for testing
# mlonde_res = 30.0
# mlatde_res = 10.
# zskip = 5
# skip = 80 



# Limit the variables to save 
vars_to_save = ['mlond', 'mlt', 'slteq', 
                'hlb', 'llb', 'Ed1', 'Ed2', 've1N', 've1S', 've2N', 've2S',
                'mlat', 'mlon', 'alt', 'time', 'mlonde', 'Phi', 'mlate', 'mlone', 'mlatde', 'mlatd', 'hAe', 'hA', 'resid_rms',]

tday = pd.to_datetime('2022-10-01') # Random day
fn = '/disks/data/icon/Repository/Archive/LEVEL.4/TIEGCM/%i/Data/ICON_L4-3_TIEGCM_%s_v02r000.NC' % (tday.year, tday.strftime('%Y-%m-%d'))
dsd = xr.open_dataset(fn)

# Use the daily-averaged altitude grid as the final altitude grid going forward
# sig from TIEGCM defined on the "lev" grid.
# wind from TIEGCM defined in the "lev" grid.
# alt from TIEGCM defined on the "ilev" grid. So I need to do something similar as the TIEGCM-MIGHTI-sampling code, so I can use regular grid interpolation.
#       I want to be sure the altitude grid of the dynamo code matches the "ilev" grid from the TIEGCM model, since that is where u and sig are defined.
# sig for dynamo defined on midpoint grid.

# Interpolate ZG to the "lev" coordinates instead of the "ilev" coordinates. Winds are defined on "lev"
dsd['ZGlev'] = dsd.ZG.interp(ilev=dsd.lev).assign_coords(lev=dsd.lev)
z = dsd.ZGlev.mean(dim=['time','lat','lon'])/1e2 # m
z = z[:-1] # Drop top altitude because it's nan
z = z[::zskip] # Downsample for testing

d = dynamo.init(z, mlonde_res = mlonde_res, mlatde_res = mlatde_res, mlatde_max = mlatde_max, tref = tday, hR=hR) # Time just affects IGRF I believe.
dynamo.get_B_IGRF(d)

d_init = dynamo.copy(d)

# Pre-load TIEGCM files to come up with "seasonal-average" dataset

Useful xr.Datasets that this creates:
- **dsday_seasavg: organized by lon, seasonal averages.**
- **dsday_seasglonavg: Same, but with geog longitudinal averaging**


# NOTE: To make this work on a separate machine you'll have to specify the correct location of the TIEGCM-ICON files on your machine

In [ ]:
# Downsampling for "seasonal average" cases.
## OPTION 1: What was used in practice. This takes a lot of memory (~30 GB)
skip_seas = 10 

## OPTION 2: Small number of files, just for testing
# skip_seas = 300

In [ ]:
tstart = datetime.now()
    
# Open a subset of TIEGCM Of all available TIEGCM files, choose ones that match the relevant doy range
dates = pd.date_range('2019-12-21','2022-11-10')[::skip_seas]

dss = []
printerase('Loading TIEGCM %i files....' % (len(dates)))
for tday in dates:
    # NOTE: Modify the location below as appropriate for your machine.
    fn = '/disks/data/icon/Repository/Archive/LEVEL.4/TIEGCM/%i/Data/ICON_L4-3_TIEGCM_%s_v02r000.NC' % (tday.year, tday.strftime('%Y-%m-%d'))
    print('\t%s'%fn)
    dsday = xr.open_dataset(fn)
    dsday = dsday[['PHIM2D', 'SIGMA_HAL', 'SIGMA_PED', 'UN', 'VN', 'ZG']] # Only keep certain variables

    # Reindex by UT hour instead of datetime
    uthr = dsday.time.dt.hour
    uthr = np.mod(uthr-1, 24)+1 # This just changes the last timestamp (0 UT) to 24, for simplicitly
    dsday['uthr'] = uthr
    dsday = dsday.swap_dims({'time':'uthr'}).drop('time')

    dsday.load() # Is this dumb?

    dss.append(dsday)
gc.collect()

In [ ]:
ds = sum(dss)/len(dss)


In [ ]:
# Now make it look like a typical TIEGCM file, by reindexing "uthr" dim to an arbitrary datetime
tbin = pd.to_datetime('2024-01-01') + pd.to_timedelta(range(1,25),unit='h') # 2024 to distinguish from 2023 in run below
ds['time'] = (['uthr'], tbin)
dsday = ds.swap_dims({'uthr':'time'}).drop('uthr')

# Create these for future use:
# dsday_seasavg: organized by lon, seasonal averages.
# dsday_seasglonavg: Same, but with geog longitudinal averaging

# And maybe or maybe not useful: dsday_seas_slt: organized by slt

dsday_seasavg = dsday
dsday_seasavg_slt = view_TIEGCM_by_slt(dsday_seasavg)
dsdayx = dsday_seasavg_slt.copy()
for v in dsdayx.keys(): # Loop over data variables

    if v == 'ZG': # Skip this; we don't want to mess with coordinate system
        continue

    if 'slt' in dsdayx[v].dims: # Replace it with the longitude-mean (which here means UT mean)
        dsdayx[v] = 0*dsdayx[v] + dsdayx[v].mean(dim='time')

dsday_seasglonavg = view_TIEGCM_by_lon(dsdayx)

dsdayx.close()
del dsdayx

In [ ]:
# OPTIONALLY USE THIS TO SAVE MEMORY
for d in dss:
    d.close()
del dss

# Run all doy bins

- This took 14 hours to run on my machine

In [ ]:
doybe = np.linspace(0,366.,9) # 46d bins to match the data. Edges of bins
garbage_collect = True # Whether to delete unused variables, etc., to save memory. Keep as True unless debugging.

doyb = (doybe[1:] + doybe[:-1])/2. # middle of bins. Be careful as this is fractional (to keep equal spacing) whereas sometimes you'll want ints.

dlist = []
tstart = datetime.now()
for i in range(len(doyb)):
    
    doy0 = int(doybe[i])
    doy1 = int(doybe[i+1])-1 # Inclusive.
    
    # Of all available TIEGCM files, choose ones that match the relevant doy range
    dates_avail = pd.date_range('2019-12-21','2022-11-10')
    doys_avail = dates_avail.dayofyear
    dates = dates_avail[(doys_avail >= doy0) & (doys_avail <= doy1)] # inclusive, as above
    dates = dates[::skip]

    dss = []
    printerase('Doys %i-%i Loading TIEGCM %i files....' % (doy0,doy1,len(dates)))
    for tday in dates:
        
        # NOTE: Modify the location below as appropriate for your machine.
        fn = '/disks/data/icon/Repository/Archive/LEVEL.4/TIEGCM/%i/Data/ICON_L4-3_TIEGCM_%s_v02r000.NC' % (tday.year, tday.strftime('%Y-%m-%d'))
        print('\t%s'%fn)
        dsday = xr.open_dataset(fn)
        dsday = dsday[['PHIM2D', 'SIGMA_HAL', 'SIGMA_PED', 'UN', 'VN', 'ZG']] # Only keep certain variables

        # Reindex by UT hour instead of datetime
        uthr = dsday.time.dt.hour
        uthr = np.mod(uthr-1, 24)+1 # This just changes the last timestamp (0 UT) to 24, for simplicitly
        dsday['uthr'] = uthr
        dsday = dsday.swap_dims({'time':'uthr'}).drop('time')

        dsday.load() # Not sure if this is necessary

        dss.append(dsday)
    gc.collect()

    ds = sum(dss)/len(dss)

    # Now make it look like a typical TIEGCM file, by reindexing "uthr" dim to an arbitrary datetime
    tbin = pd.to_datetime('2023-01-01') + pd.to_timedelta(int(doyb[i])-1, unit='day') + pd.to_timedelta(range(1,25),unit='h') # Middle of doy bin
    ds['time'] = (['uthr'], tbin) # using 2023 so that I don't mistake it for an actual mission date
    dsday = ds.swap_dims({'uthr':'time'}).drop('uthr')

    if garbage_collect:
        for d in dss:
            d.close()
        del dss

    # Create these for future use:
    # dsday_raw: organized by lon
    # dsday_glonavg: Same, but with geog longitudinal averaging

    # And maybe or maybe not useful: dsday_raw_slt: organized by slt

    dsday_raw = dsday
    dsday_raw_slt = view_TIEGCM_by_slt(dsday_raw)
    dsdayx = dsday_raw_slt.copy()
    for v in dsdayx.keys(): # Loop over data variables

        if v == 'ZG': # Skip this; we don't want to mess with coordinate system
            continue

        if 'slt' in dsdayx[v].dims: # Replace it with the longitude-mean (which here means UT mean)
            dsdayx[v] = 0*dsdayx[v] + dsdayx[v].mean(dim='time')

    dsday_lonavg = view_TIEGCM_by_lon(dsdayx)


    d_raws = []
    d_lonavgs = []
    d_seasavgs = []
    d_seasglonavgs = []
    for ti in range(len(dsday_raw.time)):
        printerase('Doys %i-%i: Ingesting TIEGCM %i' % (doy0,doy1,ti))
        t = pd.to_datetime(dsday_raw.time[ti].item())

        d = dynamo.copy(d_init)
        setup_using_TIEGCM_vars(d, dsday_raw.isel(time=ti))
        d = d.assign_coords({'time':t})
        d_raws.append(d)

        # Glon avg
        d = dynamo.copy(d_init)
        setup_using_TIEGCM_vars(d, dsday_lonavg.isel(time=ti))
        d = d.assign_coords({'time':t})
        d_lonavgs.append(d)
        
        # Seas avg
        d = dynamo.copy(d_init)
        setup_using_TIEGCM_vars(d, dsday_seasavg.isel(time=ti))
        d = d.assign_coords({'time':t})
        d_seasavgs.append(d)
        
        # Both
        d = dynamo.copy(d_init)
        setup_using_TIEGCM_vars(d, dsday_seasglonavg.isel(time=ti))
        d = d.assign_coords({'time':t})
        d_seasglonavgs.append(d)

    d_raw    = dynamo.concat_over_time(d_raws)
    d_glonavg = dynamo.concat_over_time(d_lonavgs)
    d_seasavg = dynamo.concat_over_time(d_seasavgs)
    d_seasglonavg = dynamo.concat_over_time(d_seasglonavgs)


    printerase('Doys %i-%i: Dynamo computations' % (doy0,doy1))


    # Setup cases
    d_wind_resolve_cond_resolve_hlb_resolve = d_raw # No need to copy


    #### Re-do for the different cases (longitude, seasonal averaging)
    ####
    d = dynamo.copy(d_raw)
    d[['ugN','ugS']] = d_glonavg[['ugN','ugS']]
    d = d.transpose(*list(d_raw.dims)) # Make sure ordering of dimensions is the same
    d_wind_glonavg_cond_resolve_hlb_resolve = d
    
    #### Including seasonal averaging cases.
    ####
    d = dynamo.copy(d_raw)
    d[['ugN','ugS']] = d_seasavg[['ugN','ugS']]
    d = d.transpose(*list(d_raw.dims)) # Make sure ordering of dimensions is the same
    d_wind_seasavg_cond_resolve_hlb_resolve = d
    
    ####
    d = dynamo.copy(d_raw)
    d[['ugN','ugS']] = d_seasglonavg[['ugN','ugS']]
    d = d.transpose(*list(d_raw.dims)) # Make sure ordering of dimensions is the same
    d_wind_seasglonavg_cond_resolve_hlb_resolve = d
    
    

    # Solve cases (This takes tens of sec per solve)
    d_wind_resolve_cond_resolve_hlb_resolve = dynamo.compute_FLI_solve_over_time(d_wind_resolve_cond_resolve_hlb_resolve)
    d_wind_glonavg_cond_resolve_hlb_resolve = dynamo.compute_FLI_solve_over_time(d_wind_glonavg_cond_resolve_hlb_resolve)
    d_wind_seasavg_cond_resolve_hlb_resolve = dynamo.compute_FLI_solve_over_time(d_wind_seasavg_cond_resolve_hlb_resolve)
    d_wind_seasglonavg_cond_resolve_hlb_resolve = dynamo.compute_FLI_solve_over_time(d_wind_seasglonavg_cond_resolve_hlb_resolve)
   
    
    # For all datasets you want to save, do final stuff.
    # 0. Trim the variable list to save space
    # 1. Transform to LT (easier to analyze)
    # 2. Change "time" dimension to "UT" for easier concat'ing.
    # 3. Create new dim for doy
    printerase('Doys %i-%i: Reorganizing and saving' % (doy0,doy1))
    
    dcases = [d_wind_resolve_cond_resolve_hlb_resolve,
              d_wind_glonavg_cond_resolve_hlb_resolve,
              d_wind_seasavg_cond_resolve_hlb_resolve,
              d_wind_seasglonavg_cond_resolve_hlb_resolve,
              ]
    tags =   ['wr_cr_hr',
              'wg_cr_hr',
              'ws_cr_hr',
              'wsg_cr_hr',
              ]
    
    # Save all cases over the new "driver" dimension
    dcase_vec = []
    for k in range(len(dcases)):
        
        dfull = dcases[k]
        d = dfull[vars_to_save]
        
        d = dynamo.view_by_slt(d) 

        uthr = d.time.dt.hour
        uthr = np.mod(uthr-1, 24)+1 # This just changes the last timestamp (0 UT) to 24, for simplicitly
        d['ut'] = uthr
        d = d.swap_dims({'time':'ut'})
        
        dcase_vec.append(d.assign_coords({'driver':tags[k]}))
        
        if garbage_collect:
            dfull.close()
            del dfull

    # Save this case-combined dataset, to be later combined over the "doy" dimension
    dcase_combined = dynamo.concat_over_drivers(dcase_vec)
    dlist.append(dcase_combined.assign_coords({'doy':doyb[i]})) # Note tagging dim with fractional doy
            
printerase('Almost done... concat...')
dall = dynamo.concat_over_doy(dlist)
if garbage_collect:
    # Save memory
    for d in dlist:
        d.close()
    del dlist
    gc.collect()
print('Done.')
    
tstop = datetime.now()
dt = (tstop-tstart).total_seconds()/60.
print('Total processing time: %.1f min' % (dt))

In [ ]:
if 'apex_obj' in dall.attrs: # Can't save object attributes as nc
    del dall.attrs['apex_obj']
dall.to_netcdf('tiegcm_2024_v05.nc')